****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [162]:
from lxml import etree as ET
#import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

In [163]:
doc = ET.parse('data/mondial_database.xml')

In [177]:
#ET.dump(doc.getroot())

### 10 countries with the lowest infant mortality rates

The approach is to get all the infant mortality rate elements of 'country' parent into a dataframe. 

In [165]:
root = doc.getroot()
names = []
rates = []
for country in root.findall('country') :
    names.append(country.find('name').text)
    im = country.find('infant_mortality')
    if (im == None) :
        rates.append(np.nan)
    else :
        rates.append(country.find('infant_mortality').text)
c_m = pd.DataFrame({ 'name' : names, 'Infant_Mortality' : rates})
c_m.sort_values(by='Infant_Mortality').head(10)

,Infant_Mortality,name
38,1.81,Monaco
30,10.16,Romania
153,10.2,Fiji
69,10.48,Brunei
132,10.5,Grenada
237,10.59,Mauritius
124,10.7,Panama
243,10.77,Seychelles
102,10.92,United Arab Emirates
113,10.93,Barbados


#### Utility function to get latest population

In [166]:
# A utility function which will return child 'population' element with the latest population info.
def find_latest_pop(parent) :
    pop_years = []
    for pop in parent.findall('population') :
        pop_years.append(int(pop.get('year')))
    if(len(pop_years) > 0 ) :
        max_yr = max(pop_years)
        pop_element = parent.find("./population[@year='%s']" % max_yr)
    else :
        pop_element = None
    return pop_element


In [167]:
# Testing the function for city population
ET.dump(find_latest_pop(root.find(".//city[@id='cty-Albania-Tirane']")))


<population year="2011" measured="census">418495</population>
      


In [168]:
# Testing the function for country population
ET.dump(find_latest_pop(root.find("./country[name='Albania']")))

<population year="2011" measured="census">2800138</population>
      


### 10 cities with the largest population

The approach is to get all city elements in the entire Tree into a dataframe. Some 'city' elements have a 'province' parent and some have 'country' parent. I'm also including the respective country or province name for the cities in the dataframe.

In [169]:
root = doc.getroot()
city_names = []
city_pops = []
city_cntrs = []
city_prvncs = []
for city in root.findall('.//city'):
    city_names.append(city.find('name').text)
    latest_pop = find_latest_pop(city)
    if (latest_pop == None) : # No population info
        city_pops.append(np.nan)
    else : 
        city_pops.append(int(latest_pop.text))
    if(city.get('province') == None) : # city's parent is country - no provinces
        city_cntrs.append(city.getparent().find('name').text)
        city_prvncs.append('')
    else : # get country code from the attribute in city element. Get province name from parent element
        city_cntrs.append(city.get('country'))
        city_prvncs.append(city.getparent().find('name').text)

cities = pd.DataFrame({ 'Name': city_names, 'Population': city_pops, 'Province':city_prvncs, 'Country':city_cntrs})
cities.sort_values(by='Population', ascending=False).head(10)


,Country,Name,Population,Province
1341,CN,Shanghai,22315474.0,Shanghai
771,TR,Istanbul,13710512.0,İstanbul
1527,IND,Mumbai,12442373.0,Maharashtra
479,R,Moskva,11979529.0,Moscow
1340,CN,Beijing,11716620.0,Beijing
2810,BR,São Paulo,11152344.0,São Paulo
1342,CN,Tianjin,11090314.0,Tianjin
1064,CN,Guangzhou,11071424.0,Guangdong
1582,IND,Delhi,11034555.0,Delhi
1067,CN,Shenzhen,10358381.0,Guangdong


### 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

Create a dataframe with all ethnic group entries across countries. Get the % value of the country population represented by the ethnic group. Also get country's latest population

In [170]:
root = doc.getroot()
eg_names =[]
eg_pcnt = []
cntry_names = []
cntry_pops = []
for eg in root.findall(".//ethnicgroup") :
    eg_names.append(eg.text)
    eg_pcnt.append(float(eg.get('percentage')))
    cntry_names.append(eg.getparent().find('name').text)
    pop = find_latest_pop(eg.getparent())
    if (pop == None) : # No population info
        cntry_pops = np.nan
    else :
        cntry_pops.append(float(pop.text))
egs = pd.DataFrame({'Ethnic_Group':eg_names, 'Population_Percentage' : eg_pcnt, 'Country':cntry_names, 'Country_Population': cntry_pops})
# Calculate ethnic population from country population and ethnic group percentage
egs['Ethnic_Population']= egs.apply(lambda row: 0.01 * row['Country_Population'] * row['Population_Percentage' ], axis = 1)
# Group by ethnic group and find total ethnic population across countries
egs.groupby('Ethnic_Group').sum()[['Ethnic_Population']].sort_values(by= 'Ethnic_Population', ascending = False).head(10)


,Ethnic_Population
Ethnic_Group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


### name and country of a) longest river, b) largest lake and c) airport at highest elevation

First view the structure of the xml elements for river, lake and airport. Then create appropriate data frames to query

In [171]:
ET.dump(root.find('.//river'))


<river id="river-Thjorsa" country="IS">
      <name>Thjorsa</name>
      <to watertype="sea" water="sea-Atlantic"/>
      <area>7530</area>
      <length>230</length>
      <source country="IS">
         <latitude>65</latitude>
         <longitude>-18</longitude>
      </source>
      <estuary country="IS">
         <latitude>63.9</latitude>
         <longitude>-20.8</longitude>
      </estuary>
   </river>
   


In [172]:
ET.dump(root.find('.//lake'))


<lake id="lake-Inarisee" country="SF">
      <name>Inari</name>
      <located country="SF" province="lteil-LAP-SF"/>
      <to watertype="river" water="river-Paatsjoki"/>
      <area>1040</area>
      <latitude>68.95</latitude>
      <longitude>27.7</longitude>
      <elevation>119</elevation>
      <depth>92</depth>
   </lake>
   


In [173]:
ET.dump(root.find('.//airport'))


<airport iatacode="HEA" city="cty-Afghanistan-2" country="AFG">
      <name>Herat</name>
      <latitude>34.210017</latitude>
      <longitude>62.2283</longitude>
      <elevation>977</elevation>
      <gmtOffset>5</gmtOffset>
   </airport>
   


#### Longest River

In [174]:
root = doc.getroot()
cntry_names = []
river_names =[]
river_len = []
for river in root.findall('.//river') :
    cntry_names.append(river.get('country'))
    river_names.append(river.find('name').text)
    length = river.find('length')
    if (length == None) :
        river_len.append(np.nan)
    else :
        river_len.append(float(length.text))
rivers = pd.DataFrame({'Country': cntry_names, 'River_Name' : river_names, 'River_Length': river_len})
rivers.sort_values(by = 'River_Length', ascending=False).head(1)

,Country,River_Length,River_Name
174,CO BR PE,6448.0,Amazonas


#### Largest Lake

In [175]:
root = doc.getroot()
cntry_names = []
lake_names =[]
lake_area = []
for lake in root.findall('.//lake') :
    cntry_names.append(lake.get('country'))
    lake_names.append(lake.find('name').text)
    length = lake.find('area')
    if (length == None) :
       lake_area.append(np.nan)
    else :
        lake_area.append(float(length.text))
lakes = pd.DataFrame({'Country': cntry_names, 'Lake_Name' : lake_names, 'Lake_Area': lake_area})
lakes.sort_values(by = 'Lake_Area', ascending=False).head(1)

,Country,Lake_Area,Lake_Name
54,R AZ KAZ IR TM,386400.0,Caspian Sea


#### Airport at highest elevation

In [176]:
root = doc.getroot()
cntry_names = []
aprt_names =[]
aprt_elev = []
for aprt in root.findall('.//airport') :
    cntry_names.append(aprt.get('country'))
    aprt_names.append(aprt.find('name').text)
    length = aprt.find('elevation')
    if ((length == None) | (length.text == None)) :
       aprt_elev.append(np.nan)
    else :
        aprt_elev.append(float(length.text))
lakes = pd.DataFrame({'Country': cntry_names, 'Airport_Name' : aprt_names, 'Airport_Elevation': aprt_elev})
lakes.sort_values(by = 'Airport_Elevation', ascending=False).head(1)

,Airport_Elevation,Airport_Name,Country
80,4063.0,El Alto Intl,BOL
